In [27]:
from pyspark.sql import SparkSession
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import  StringIndexer, VectorAssembler, VectorIndexer, OneHotEncoder, MinMaxScaler
from pyspark.ml.linalg import Vectors
#from pyspark.ml.datatypes import  StringType , IntegerType
from pyspark.sql.types import *



spark = SparkSession.builder.appName('ml-bank').getOrCreate()
df = spark.read.csv('ibm-hr-analytics-attrition.csv', header = True, inferSchema = True)

df = df.selectExpr("PerformanceRating as label","Age","Attrition","JobSatisfaction","OverTime","Gender")
df.count()

1470

In [28]:
df = df.na.drop()
df.toPandas()

,label,Age,Attrition,JobSatisfaction,OverTime,Gender
0,3,41,Yes,4,Yes,Female
1,4,49,No,2,No,Male
2,3,37,Yes,3,Yes,Male
3,3,33,No,3,Yes,Female
4,3,27,No,2,No,Male
...,...,...,...,...,...,...
1465,3,36,No,4,No,Male
1466,3,39,No,1,No,Male
1467,4,27,No,2,Yes,Male
1468,3,49,No,2,No,Male


In [32]:
categorical_variables = [ "Attrition",  "OverTime", "Gender"]
numerical_variables = ["Age" , "JobSatisfaction"]

indexers = [StringIndexer(inputCol=column, outputCol=column+"-index") for column in categorical_variables]
encoder = OneHotEncoder(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCols=["{0}-encoded".format(indexer.getOutputCol()) for indexer in indexers])
scalers = [MinMaxScaler(inputCol=column, outputCol= column+"-scaled") for column in numerical_variables]
assembler = VectorAssembler(inputCols= encoder.getOutputCols() + [scaler.getOutputCol() for scaler in scalers] , outputCol="features")

stages = []
stages += indexers
stages += [encoder]
stages += scalers
stages += [assembler]
model_preprocessor = Pipeline(stages = stages)


In [30]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=100)
model_predictor =  Pipeline(stages = [lr])

In [33]:
train, test = df.randomSplit([0.8,0.2],seed=12345)
train.printSchema()

root
 |-- label: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- OverTime: string (nullable = true)
 |-- Gender: string (nullable = true)



In [34]:
model_preprocessor = model_preprocessor.fit(train)

IllegalArgumentException: requirement failed: Column Age must be of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually int.

In [ ]:
train, test = df.randomSplit([0.8,0.2],seed=12345)

In [ ]:
model_train_data  = model_preprocessor.transform(train)
model_train_data.toPandas()


In [10]:
model_predictor = model_predictor.fit(model_train_data)

0.16397228637413394

In [ ]:
# Test


In [ ]:

labels =  result.toPandas()['label'].values
labels = np.array(labels)

pred_values = result.toPandas()['prediction'].values
pred_values = np.array(pred_values)

errr = np.sqrt(labels - pred_values).mean()
errr

In [11]:
item =  df.first()
item

Row(label=3, Age=41, Attrition='Yes', JobSatisfaction=4, OverTime='Yes', Gender='Female')

In [12]:
result = pipeline.fit(item).transform(item)
result

AttributeError: _jdf